#Created By: Rahul BASU 
#On: 2018-09-13
#Bokeh installation
    #https://bokeh.pydata.org/en/latest/docs/installation.html
    #Run Anaconda prompt as system administrator 
        conda install bokeh
    #Verify installation:
        import bokeh
        bokeh.__version__
    #Test bokeh
        testplot = figure(plot_width=400, tools='pan,box_zoom')
        testplot.circle([1,2,3,4,5], [8,6,5,2,3])
        output_file('circle.html')
        show(testplot)

In [1]:
# Perform necessary imports for pandas & bokeh & test
import pandas as pd
from bokeh.io import output_file, show, curdoc
from bokeh.plotting import figure
from bokeh.layouts import widgetbox,row
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper,Slider,MultiSelect,Select
from bokeh.palettes import magma,inferno

In [2]:
#Import csv stock data to pandas dataframe
#Dataset Source:https://www.kaggle.com/sugandhkhobragade/data-daily-market-price-20082018
data=pd.read_csv('BSE_30.csv')
data['Date']=pd.to_datetime(data['Date'])

In [ ]:
#Initial Data Exploration
#data.shape
#data.info()
#data.describe()
#data.head()
#list_of_stocks=data['Symbol'].unique()
#print(list_of_stocks)

In [ ]:
#Manipulating Data Frame for INFY & TCS
#data = data[(data.Symbol == 'INFY') | (data.Symbol == 'TCS')]

# Altering indexes to Symbol & Date - to be done if required. Corresponding plotting code has to be changed.
#data = data.set_index(['Date']).sort_index()
#data=data.loc['INFY':'TCS']
#data.head()

Non-Interactive Basic Plotting

In [ ]:
# Make the ColumnDataSource: source
source = ColumnDataSource(data={
    'x':data.Date,
    'y':data.Close,
    'symbol':data.Symbol,
    'open' :data.Open,
    'high' :data.High,
    'low' :data.Low,
    'adjclose': data.AdjClose,
    'volume': data.Volume
})

xmin, xmax = min(data.Date),max(data.Date)
ymin, ymax = min(data.Close),max(data.Close)

 # Create the figure: plot
plot = figure(title='BSE 30',
              x_range=(xmin,xmax),y_range=(ymin,ymax),
              plot_height=800,plot_width=1100)

#Unique list of Symbols & Color Mapper
stock_list = data.Symbol.unique().tolist()
color_mapper = CategoricalColorMapper(factors=stock_list, palette=inferno(len(stock_list)))

plot.circle(x='x',y='y',fill_alpha=0.8,source=source, 
                color=dict(field='symbol', transform=color_mapper), hover_fill_color='firebrick', hover_alpha=0.1, 
               legend='symbol')
plot.legend.location = 'top_right'

# Setting the axis labels
plot.xaxis.axis_label = 'Date'
plot.yaxis.axis_label = 'Close'

hover = HoverTool(tooltips=[('Symbol','@symbol')])
plot.add_tools(hover)

output_file('bse30_basic.html')
show(plot)

Add Interactivity
Interactivity Requirements:
    y- axis can change (single - select then multi-select) 
    Date Time remains as x-axis
    Slider for date time
    stock can change (single - select then multi-select) 

In [3]:
#Define callback function for interactivity
def update_plot(attr, old, new):
    
    #Read value of dropdown for y axis
    y = y_select.value
     
    new_data = {
            'x':data.Date,
            'y':data.y,
            'symbol':data.Symbol,
            'open' :data.Open,
            'high' :data.High,
            'low' :data.Low,
            'close' :data.Close,
            'adjclose': data.AdjClose,
            'volume': data.Volume
    }
    
    source.data = new_data
    
    #Set range for axes
    xmin, xmax = min(data.Date),max(data.Date)
    plot.y_range.start = min(data[y])
    plot.y_range.end = max(data[y])
    
    #Label plot axes
    plot.xaxis.axis_label = 'Date'
    plot.yaxis.axis_label = y
   
    #Unique list of Symbols & Color Mapper
    stock_list = data.Symbol.unique().tolist()
    color_mapper = CategoricalColorMapper(factors=stock_list, palette=inferno(len(stock_list)))
    
    plot.circle(x='x',y='y',fill_alpha=0.8,source=source, 
                color=dict(field='symbol', transform=color_mapper), hover_fill_color='firebrick', hover_alpha=0.1, 
               legend='symbol')
    plot.legend.location = 'top_right'
    
    hover = HoverTool(tooltips=[('Symbol','@symbol')])
    plot.add_tools(hover)

    # Add title to figure: plot.title.text
    #plot.title.text = 'Plotting for %s' % y

In [7]:
# Create the figure: plot
plot = figure(title='BSE 30',
               plot_height=800,plot_width=1100)

# Create dropdown Select widget for the y data: y_select
y_select = Select(
    options=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
    value='close',
    title='y-axis data'
)

# Attach the update_plot callback to the 'value' property of y_select
y_select.on_change('value',update_plot)

In [8]:
# Create layout and add to current document
layout = row(widgetbox(y_select), plot)
curdoc().add_root(layout)

In [9]:
output_file('bse30_interactive.html')
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [ ]:
# # Create a dropdown Select widget for Stock
# stock_select=Select(options=stock_list,value='ADANIPORTS',title='Stock')
# stock_select.on_change('value',update_plot)

# # Create layout and add to current document
# layout = row(widgetbox(stock_select), plot)
# curdoc().add_root(layout)

In [ ]:
#Multi Select
#multi_select = MultiSelect(title="Stock(s):", value=["TCS"],
#                          options=[("INFY","INFY"),("TCS","TCS")])
#show(multi_select)

#Dropdown
#menu = [("INFY", "infy"), ("TCS", "TCS")]
#dropdown = Dropdown(label="Dropdown button", button_type="warning", menu=menu)
#show(dropdown)

In [ ]:
# Make slider object: slider
# slider = Slider(start=2008,end=2018,step=1,value=2008,title='Year')
# Attach the callback to the 'value' property of slider
#slider.on_change('value',update_plot)

# Make a row layout of widgetbox(slider) and plot and add it to the current document
#layout = row(widgetbox(slider), plot)
#curdoc().add_root(layout)